## Pregunta 1
Desarrollar el algoritmo minimax para el juego 3 en raya

In [1]:
import numpy as np
import math

In [3]:
# Win representa una lista con la formas de ganar, en total
# hay 8 maneras de que alguien pueda ganar
win = [(np.array([0, 1, 2]), np.array([0, 0, 0])),
 (np.array([0, 1, 2]), np.array([1, 1, 1])),
 (np.array([0, 1, 2]), np.array([2, 2, 2])),
 (np.array([0, 0, 0]), np.array([0, 1, 2])),
 (np.array([1, 1, 1]), np.array([0, 1, 2])),
 (np.array([2, 2, 2]), np.array([0, 1, 2])),
 (np.array([0, 1, 2]), np.array([0, 1, 2])),
 (np.array([0, 1, 2]), np.array([2, 1, 0]))
 ]

In [124]:
def Heuristica(initial, player1, player2):
    # la variable count cuenta el nro. de filas, columnas y diagonales
    # completas disponibles para player1
    count = 0
    for i in range(8): # por cada posibilidad de ganar
        empty = np.array(['']*3) ==  np.array(initial[win[i]])
        # si player1 tiene fila, columna o diaginal libre (player2 no debe estar) o
        # si existen filas, columnas o diagonales vacías, entonces aumenta el conteo
        if (player1 in initial[win[i]] and player2 not in initial[win[i]]) or empty.all():
            count +=1
    return count

def Utilidad(state): # La función utilidad del estado
    # Cuando algun jugador gana ya no se halla la funcion utilidad de manera usual
    for i in range(8):
        # ful_X es vector de booleanos que compara a la lista de X con las posibles maneras de ganar
        full_X = np.array(['X']*3) ==  np.array(initial[win[i]])
        # Si es que jugador X ganó, entonces se representa como +infty
        if full_X.all() == True:
            return math.inf
        # ful_O es vector de booleanos que compara a la lista de O con las posibles maneras de ganar
        full_O = np.array(['O']*3) ==  np.array(initial[win[i]])
        # Si es que jugador O ganó, entonces se representa como +infty
        if full_O.all() == True:
            return -math.inf
    # Cuando nungino gana entonces devuelve:
    # [número de filas, columnas y diagonales completas disponibles para lugador1] –
    # [número de filas, columnas y diagonales completas disponibles para MIN]
    return Heuristica(state, 'X', 'O') - Heuristica(state, 'O', 'X')

In [126]:
# Esta función acumula los estados recorridos en la lista Closed, aqui consideramos 2 cosas
# - Simetria: Pasar al estado 'state' es equivalente a pasar a 'state' transpuesto
# - Rotación: Pasar al estado 'state' es equivalente a pasar a 'state' rotado 90, 180 y 270

def Add_to_closed(Closed,state_1):
    Closed.append(state_1)
    transpose = state_1.T
    Closed.append(transpose)
    rotate = np.rot90(state_1)
    for k in range(3):
        Closed.append(rotate)
        rotate = np.rot90(rotate)
    return list(np.unique(Closed, axis=0))  # Al final si algun estado se repite lo consideramos solo una vez

# Funci[on que obtiene las posibles jugadas siguientes
# -> Closed: el conjunto cerrado
# -> initial: estado actual
# -> current: ficha que va a jugar
def Get_children(Closed, initial, current):
    # Consideramos los indices de los espacios vacios (posibles opciones de poner la siguiente ficha)
    (inx_l,iny_l) =np.where(initial=='')

    # children almacena los nodos hijos del estado 'initial'
    children = []

    for i in range(len(inx_l)):
        initial_2 = initial.copy()
        # Asignamos a la ficha que jugara en todos los posibles espacios en blanco
        initial_2[inx_l[i],iny_l[i]] = current

        # valid es una lista de arrays en 2d con tipo de dato booleano
        valid = initial_2 == Closed

        # Si la suma de elementos de los elementos de valid es 9 significa que
        # el estado initial2 está dentro del conjunto cerrados, lo que deseamos
        # es que se considere un nuevo hijo si es que el estado initial2 no está
        # dentro de Closed; es decir, la suma no es 9
        if 9 not in [valid[k].sum() for k in range(len(valid))]: # si no esta en Closed
            children.append(initial_2)

        # Luego de tener los nodos hijo actualizamos nuestro conjunto cerrado
        Closed = Add_to_closed(Closed,initial_2)
    return Closed, children

In [125]:
def Minimax(state, depth, is_maximizing, Closed):

    # Score es el resultado de la funcion utilidad aplicada al estado actual
    score = Utilidad(state)

    # minimax viene a ser una función recursiva que se detendrá siempre y cuando
    # -> ganó el jugador1
    # -> ganó el jugador2
    # -> la profundidad es 0 (estamos en la produndidad limite y
    #       ya no podemos continuar recorriendo los hijos del estado)

    if score == math.inf or score == -math.inf or depth == 0:
        return score, 0 # el 0 es un valor por default

    # Si está jugando Max
    if is_maximizing:
        # Obtenemos los hijos de 'state'
        Children = Get_children(Closed, state, 'X')[1]
        # Comenzamos con un valor en Eval, para no tener vacío el indice del máximo
        Eval = [-math.inf]
        # Para cada hijo de state
        for n_child in range(len(Children)):
            # -> Recogemos el máximo valor de los hijos (que son el mínimo valor de
            # sus hijos y así consecutivamente) y vemos en que indice de los hijos se dió el máximo
            # eso nor sirve al final de las iteraciones (cuando lleguemos al estado inicial)
            # para saber que jugada siguiente se debe dar (se debe dar donde se dió el máximo)
            # -> Argumento False: le toca jugar a O
            eval = Minimax(Children[n_child],depth - 1, False, Closed)[0] # solo necesitamos el score
            # Eval almacena los scores que recoge de cada hijo
            Eval.append(eval)
        return max(Eval), Eval.index(max(Eval))-1 # el indice es sin considerar el -infty inicial
    else:
        # De la misma manera si is_maximizing == False, que significa que le
        # toca jugar al jugador 'O'
        Children = Get_children(Closed,state, 'O')[1]
        Eval = [math.inf]
        for n_child in range(len(Children)):
            eval = Minimax(Children[n_child],depth - 1, True, Closed)[0]
            Eval.append(eval)
        return min(Eval), Eval.index(min(Eval))-1

Ejemplo detallado en el PDF

In [109]:
# Comenzamos con un estado inicial de un tablero Tic, Tac Toe vacio
initial = np.zeros((3,3), dtype = np.dtype("<U1"))
# Llenemos un poco el tablero
initial[(0,1),(0,1)] = 'X'
initial[(0,2,2),(1,1,2)] = 'O'
initial

array([['X', 'O', ''],
       ['', 'X', ''],
       ['', 'O', 'O']], dtype='<U1')

In [110]:
Closed = [initial]
(score, child) = Minimax(initial, 3, True, Closed)

In [111]:
print('El movimieto del oponente es:')
Get_children([initial],initial, 'X')[1][child]

El movimieto del oponente es:


array([['X', 'O', ''],
       ['', 'X', ''],
       ['X', 'O', 'O']], dtype='<U1')

In [112]:
# Comenzamos con un estado inicial de un tablero vacio, para el caso contrario de fichas
initial = np.zeros((3,3), dtype = np.dtype("<U1"))
initial[(0,1),(0,1)] = 'O'
initial[(0,2,2),(1,1,2)] = 'X'
initial

array([['O', 'X', ''],
       ['', 'O', ''],
       ['', 'X', 'X']], dtype='<U1')

In [113]:
(score, child) = Minimax(initial, 3, False, [initial])

In [114]:
print('El movimieto del oponente es:')
Get_children([initial],initial, 'O')[1][child]

El movimieto del oponente es:


array([['O', 'X', ''],
       ['', 'O', ''],
       ['O', 'X', 'X']], dtype='<U1')

### Juego simulado

In [115]:
# Quien comienza primero
first = ''
while first != 'YES' and first != 'NO':
    first = input('First to start? (Yes or No): ').upper()

# Quien será el usuario
user = 'X'
is_maximizing = False
maquina = 'O'

# Comienza el tablero vacío
initial = np.zeros((3,3), dtype = np.dtype("<U1"))
Closed = [initial]

(ind_x,ind_y) = np.where(initial=='')

if first == 'YES': # si comienza primero entonces introducirá su jugada,
                   # de otra manera el oponente comienza
    user_turn = True
else:
    user_turn = False

# mientras haya espacio vacío en el tablero
while len(ind_x)>0:
    if user_turn == True:
        pos = [] # posicion inicial por default
        # Si no esta dentro de los valores permitidos o la posición que coloque esté ya ocupada
        # entonces seguira pidiendo la posicion de la jugada del usuario
        while len(pos) != 2 or (pos > [2]*2).all(0) or (pos < [0]*2).all() or not lleno:
            pos = np.array(list(map(int, input("Posicion donde quieres colocar tu ficha: ").split())))
            lleno = np.array([(pos[0],pos[1]) == (ind_x[i],ind_y[i]) for i in range(len(ind_x))]).any()
        # Introducimos la jugada del usuario
        initial[pos[0],pos[1]] = user
        print("Tu jugada:")
        print(initial)

        # Si la utilidad toma in ¡infty es porque X (el usuario) ya completó su jugana ganadora
        if Utilidad(initial) == math.inf:
            print('Ganaste el juego!!')
            break
        # pasamos al turno de la maquina
        user_turn = False

    if user_turn == False:
        # 3 es la profundidad, is?maximizing es False porque le toca al O
        (score, child) = Minimax(initial, 3, is_maximizing, Closed)
        print('Movimiento del oponente:')
        Closed_2 = [initial]
        # Ahora la siguiente jugada será nuestro nuevo tablero (initial)
        initial = Get_children(Closed_2, initial, maquina)[1][child]
        print(initial)

        # Si en la jugada de la máquina la utilidad nos da 'infty significa que
        # la máquina alcanzó si jugada ganadora, o sea perdimos el juego
        if Utilidad(initial) == -math.inf:
            print('Perdiste el juego!!')
            break
        user_turn = True

    # Almacena los indices de los epspacios en blanco
    (ind_x,ind_y) = np.where(initial=='')

First to start? (Yes or No): no
Movimiento del oponente:
[['' '' '']
 ['' 'O' '']
 ['' '' '']]
Posicion donde quieres colocar tu ficha: 0 0
Tu jugada:
[['X' '' '']
 ['' 'O' '']
 ['' '' '']]
Movimiento del oponente:
[['X' 'O' '']
 ['' 'O' '']
 ['' '' '']]
Posicion donde quieres colocar tu ficha: 2 1
Tu jugada:
[['X' 'O' '']
 ['' 'O' '']
 ['' 'X' '']]
Movimiento del oponente:
[['X' 'O' 'O']
 ['' 'O' '']
 ['' 'X' '']]
Posicion donde quieres colocar tu ficha: 2 1
Posicion donde quieres colocar tu ficha: 2 0
Tu jugada:
[['X' 'O' 'O']
 ['' 'O' '']
 ['X' 'X' '']]
Movimiento del oponente:
[['X' 'O' 'O']
 ['O' 'O' '']
 ['X' 'X' '']]
Posicion donde quieres colocar tu ficha: 2 2
Tu jugada:
[['X' 'O' 'O']
 ['O' 'O' '']
 ['X' 'X' 'X']]
Ganaste el juego!!


In [123]:
## 2da prueba del código de arriba

First to start? (Yes or No): no
Movimiento del oponente:
[['' '' '']
 ['' 'O' '']
 ['' '' '']]
Posicion donde quieres colocar tu ficha: 2 0
Tu jugada:
[['' '' '']
 ['' 'O' '']
 ['X' '' '']]
Movimiento del oponente:
[['O' '' '']
 ['' 'O' '']
 ['X' '' '']]
Posicion donde quieres colocar tu ficha: 2 2
Tu jugada:
[['O' '' '']
 ['' 'O' '']
 ['X' '' 'X']]
Movimiento del oponente:
[['O' '' '']
 ['' 'O' '']
 ['X' 'O' 'X']]
Posicion donde quieres colocar tu ficha: 1 2
Tu jugada:
[['O' '' '']
 ['' 'O' 'X']
 ['X' 'O' 'X']]
Movimiento del oponente:
[['O' 'O' '']
 ['' 'O' 'X']
 ['X' 'O' 'X']]
Perdiste el juego!!
